### Añadimos SRC a la raíz del proyecto para poder importar el contenido

In [ ]:
import sys
import os

# Obtener la ruta absoluta de la carpeta raíz (donde está src)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde notebooks/

# Agregar la carpeta src al path
sys.path.append(os.path.join(ROOT_DIR, "src"))

### Importamos los módulos necesarios

In [ ]:
import pandas as pd

from config import VICTORY_ROAD_SCRAPPING_DATA_PATH, POKEMON_DATASET_FULL_MERGED, SMOGON_MISSING_IN_VICTORY_ROAD, VICTORY_ROAD_MISSING_IN_SMOGON

### Borramos los registros de Vivillon y Alcremie, ya que son conflictivos para la integración en el dataset y no son demasiado relevantes los datos que hay de ellos

In [ ]:
# 📂 Cargar el dataset de Worlds VGC
df_worlds = pd.read_csv(VICTORY_ROAD_SCRAPPING_DATA_PATH)

# 📌 1. Convertir nombres a minúsculas para evitar problemas de mayúsculas/minúsculas
df_worlds["Pokemon"] = df_worlds["Pokemon"].str.lower()

# 📌 2. Filtrar para eliminar Pokémon que contengan "vivillon" o "alcremie"
df_worlds_filtered = df_worlds[~df_worlds["Pokemon"].str.contains("vivillon|alcremie", regex=True)]

# 📊 Mostrar cuántas filas se eliminaron
removed_count = len(df_worlds) - len(df_worlds_filtered)
print(f"✅ Eliminados {removed_count} registros de 'vivillon' y 'alcremie'.")

# 📂 Guardar el dataset actualizado
df_worlds_filtered.to_csv(VICTORY_ROAD_SCRAPPING_DATA_PATH_CLEANED, index=False)
print("📁 Dataset actualizado guardado en '{VICTORY_ROAD_SCRAPPING_DATA_PATH_CLEANED}.")

### Comparamos los resultados que hemos obtenido en el scraping con nuestro dataset de todos los pokemon y vemos si hay diferencias en los nombres de los pokemon para poder hacer correctamente el merge 

In [ ]:
# 📂 Cargar los datasets
df_main = pd.read_csv(POKEAPI_DATASET_PATH_WITH_SMOGON)  # Dataset principal (ya tiene Smogon)
df_worlds = pd.read_csv(VICTORY_ROAD_SCRAPPING_DATA_PATH_CLEANED)  # Dataset con datos de Worlds VGC

# 🔍 Verificar nombres de columnas
print("🔎 Columnas en df_main:", df_main.columns.tolist())
print("🔎 Columnas en df_worlds:", df_worlds.columns.tolist())

# 📌 1. LIMPIEZA Y NORMALIZACIÓN DE NOMBRES
def clean_name(name):
    """Función para limpiar nombres de Pokémon (minúsculas, sin espacios, sin caracteres conflictivos)"""
    return (
        name.lower()
        .replace(".", "")  # Eliminar puntos
        .replace("'", "")  # Eliminar apóstrofes
        .replace(" ", "-")  # Reemplazar espacios por guiones
        .strip()
    )

# Aplicar limpieza a los nombres en ambos datasets
df_main["name"] = df_main["name"].astype(str).apply(clean_name)
df_worlds["Pokemon"] = df_worlds["Pokemon"].astype(str).apply(clean_name)

# 📌 2. COMPARACIÓN ENTRE LOS DOS DATASETS
smogon_names = set(df_main["name"])  # Nombres en el dataset principal (con Smogon)
worlds_names = set(df_worlds["Pokemon"])  # Nombres en el dataset de Worlds VGC

# ✅ Coincidencias
matched_names = smogon_names & worlds_names  # Nombres que coinciden
unmatched_smogon = smogon_names - worlds_names  # Nombres en Smogon que NO están en Worlds
unmatched_worlds = worlds_names - smogon_names  # Nombres en Worlds que NO están en Smogon

# 📊 Mostrar estadísticas
print(f"🔍 Total de nombres en Smogon: {len(smogon_names)}")
print(f"🔍 Total de nombres en Worlds VGC: {len(worlds_names)}")
print(f"✅ Coincidencias entre ambos datasets: {len(matched_names)}")
print(f"⚠️ Pokémon en Smogon que no están en Worlds: {len(unmatched_smogon)}")
print(f"⚠️ Pokémon en Worlds que no están en Smogon: {len(unmatched_worlds)}")

# 📁 Guardar listas de Pokémon huérfanos para revisión manual
pd.DataFrame({"Unmatched_Smogon": list(unmatched_smogon)}).to_csv(SMOGON_MISSING_IN_VICTORY_ROAD, index=False)
pd.DataFrame({"Unmatched_Worlds": list(unmatched_worlds)}).to_csv(VICTORY_ROAD_MISSING_IN_SMOGON, index=False)

print("📁 Listado de nombres huérfanos guardado en '{SMOGON_MISSING_IN_VICTORY_ROAD}' y '{VICTORY_ROAD_MISSING_IN_SMOGON}'.")

### Una vez que tenemos todos los pokemon coincidentes por el nombre, realizamos el Merge con el dataset principal

In [ ]:
# 📂 Cargar datasets
df_official = pd.read_csv(POKEAPI_DATASET_PATH_WITH_SMOGON)  # Dataset principal
df_worlds = pd.read_csv(VICTORY_ROAD_SCRAPPING_DATA_PATH_CLEANED)  # Datos de Worlds VGC

# 📌 Fusionar datasets por el nombre del Pokémon
df_merged = df_official.merge(df_worlds, left_on="name", right_on="Pokemon", how="left")

# 📌 Eliminar la columna redundante "Pokemon" (ya está en "name")
df_merged.drop(columns=["Pokemon"], inplace=True)

# 📌 Rellenar valores nulos en columnas de Worlds VGC con "NoUsage"
worlds_columns = ["Worlds2022Usage", "Worlds2023Usage", "Worlds2024Usage"]
df_merged[worlds_columns] = df_merged[worlds_columns].fillna("NoUsage")

# 📂 Guardar el dataset final fusionado

df_merged.to_csv(POKEMON_DATASET_FULL_MERGED, index=False)

# 📊 Mostrar resultado
print(f"✅ Dataset fusionado correctamente y guardado en: {POKEMON_DATASET_FULL_MERGED}")
df_merged.head()

### Renombramos las columnas para que sea mas clara la diferenciacion de los datos de Smogon con los datos de los Worlds de VGC

In [ ]:
# 📂 Cargar el dataset combinado
df = pd.read_csv(POKEMON_DATASET_FULL_MERGED)

# 📌 Diccionario de nuevos nombres
column_rename_map = {
    "VGCusage2022": "Smogon_VGC_Usage_2022",
    "VGCusage2023": "Smogon_VGC_Usage_2023",
    "VGCusage2024": "Smogon_VGC_Usage_2024",
    "Worlds2022Usage": "Worlds_VGC_Usage_2022",
    "Worlds2023Usage": "Worlds_VGC_Usage_2023",
    "Worlds2024Usage": "Worlds_VGC_Usage_2024"
}

# 📌 Renombrar columnas
df.rename(columns=column_rename_map, inplace=True)

# 📊 Verificar cambios
print("✅ Columnas renombradas correctamente:")
print(df.head())

# 📂 Guardar el dataset actualizado
df.to_csv(POKEMON_DATASET_FULL_MERGED, index=False)
print("📁 Dataset actualizado guardado en 'pokemon_dataset_full_merged.csv'.")

### Rellenamos los valores nulos de type2 y ability1 con los valores "No_type" y "No_ability"  

In [ ]:
# 📂 Cargar el dataset con VGC integrado
df_merged = pd.read_csv(POKEMON_DATASET_FULL_MERGED)

# 🛠️ Rellenar valores nulos en Type2 y ability2 con "None"
df_merged["type2"] = df_merged["type2"].fillna("No_type")
df_merged["ability2"] = df_merged["ability2"].fillna("No_ability")

# 📊 Verificar que no haya valores nulos restantes
missing_values = df_merged[["type2", "ability2"]].isnull().sum()
print(f"🔎 Valores nulos restantes después del relleno:\n{missing_values}")

# 📁 Guardar el dataset final
df_merged.to_csv(POKEMON_DATASET_FULL_MERGED, index=False)

print(f"✅ Dataset final guardado en '{output_path}'")

In [ ]:
df_merged.info()